In [ ]:
allOfStaffs = None
careStaff_jokin = None
careStaff_hijokin = None
nightShift = None
careStaffKaifuku_jokin = None
careStaffKaifuku_hijokin = None
careStaffRate_5yearsExp = None
careStaffRate_10yearsExp = None
careStaffQuit_jokin = None
careStaffQuit_hijokin = None
Nurse_jokin = None
Nurse_hijokin = None
NurseQuit_jokin = None
NurseQuit_hijokin = None
careStaffNurseQuit_jokin = None
careStaffNurseQuit_hijokin = None
nursePracticalNurse_jokin = None
nursePracticalNurse_hijokin = None
nursePracticalNurseQiut_jokin = None
nursePracticalNurseQiut_hijokin = None
homeHelper_jokin = None
homeHelper_hijokin = None
homeHelperQuit_jokin = None
homeHelperQuit_hijokin = None
homeHelperKaifuku_jokin = None
homeHelperKaifuku_hijokin = None
homeHelperRate_5yearsExp = None
homeHelperRate_10yearsExp = None
staffQuit_jokin = None
staffQuit_hijokin = None
careMgr_jokin = None
careMgr_hijokin = None
shuninCareMgr_jokin = None
shuninCareMgr_hijokin = None
careMgrNurse_jokin = None
careMgrNurse_hijokin = None
careMgrkaifuku_jokin = None
careMgrkaifuku_hijokin = None
careMgrQuit_jokin = None
careMgrQuit_hijokin = None
careMgrRate_10yearsExp = None
planner_jokin = None
planner_hijokin = None
WelEquipSpecialist_jokin = None
WelEquipSpecialist_hijokin = None
WelEquipSpecialistQuit_jokin = None
WelEquipSpecialistQuit_hijokin = None
WelEquipSpecialistRate_10yearsExp = None
publicHealthNurse_jokin = None
publicHealthNurse_hijokin = None
publicHealthNurseNursePracticalNurseQiut_jokin = None
publicHealthNurseNursePracticalNurseQiut_hijokin = None
publicHealthNurseNursePracticalNurseQiutRate_5yearsExp = None
operator_jokin = None
operator_hijokin = None
operatorQuit_jokin = None
operatorQuit_hijokin = None
PTOTST_jokin = None
PTOTST_hijokin = None
PTOTSTQuit_jokin = None
PTOTSTQuit_hijokin = None
PTOTSTRate_10yearsExp = None

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from selenium.webdriver.chrome.options import Options
import pandas as pd
import random
import math

# ヘッドレス、シークレットモード、ユーザーエージェントをランダムで変更
options = Options()
options.add_argument('incognito')
# options.add_argument('--headless')
user_agent = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.2 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36']

ua = user_agent[random.randrange(0, len(user_agent), 1)]
options.add_argument('--user-agent=' + ua)

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# 暗黙的待機　要素を取得するまで最大引数秒分待機してね。
# driver.implicitly_wait(10)

# 最大の読み込み時間を設定 今回は最大30秒待機できるようにする
wait_ = WebDriverWait(driver=driver, timeout=30)
# 要素が全て検出できるまで待機する
wait = wait_.until(EC.presence_of_all_elements_located)

In [ ]:
# 都道府県番号を指定して該当ページにアクセス
pref = 23
url = f'https://www.kaigokensaku.mhlw.go.jp/{pref}/index.php'
driver.get(url)

wait
driver.find_element(by=By.CSS_SELECTOR, value='#searchselect_centerMenu1 > a > img').click()

wait
driver.find_element(by=By.CSS_SELECTOR, value='#osusumeMenu2').click()

wait

# この順番でクリックしないとモーダル出てきてうざい
for number in range(1,8,2):  
    service = driver.find_element(by=By.CSS_SELECTOR, value=f'#selectServiceBlock > ul > li:nth-child({number})').click()

for number in range(2,7,2):  
    service = driver.find_element(by=By.CSS_SELECTOR, value=f'#selectServiceBlock > ul > li:nth-child({number})').click()

driver.find_element(by=By.CSS_SELECTOR, value='#areaSelectTabs > li:nth-child(2) > a').click()
driver.find_element(by=By.CSS_SELECTOR, value='#checkAllAreaList').click()
driver.find_element(by=By.CSS_SELECTOR, value='#ProposeSearchForm > div:nth-child(8) > div.text-center > a').click()

In [ ]:
# 一覧ページでの要素取得と遷移

sleep(3)
data = []
staffInfoData = []

# ページ数の調査
items_ = driver.find_element(by=By.CSS_SELECTOR, value='#sel_ResultListForm > div.tab-content > nav:nth-child(1) > ul:nth-child(1) > li > span.red > b').text
items = items_.replace('件','')
allOfItems = math.floor(int(items)/50)

# 表示件数を5件→50件に変更
driver.find_element(by=By.CSS_SELECTOR, value='#displayNumber > option:nth-child(4)').click()

for i in range(allOfItems):
    wait
    sleep(3)
    
    # 事業所の詳細ページに遷移
    resultLists = driver.find_element(by=By.CSS_SELECTOR, value='#resultList')
    lists = resultLists.find_elements(by=By.LINK_TEXT, value='詳細情報を見る')

    links = [i.get_attribute('href') for i in lists]

    for link in links:
        driver.get(link)
        table = []

        wait
        # 事業所名の取得
        name = driver.find_element(by=By.CSS_SELECTOR, value='#jigyosyoName').text

        wait
        # サービス種別
        tableTexts = driver.find_element(by=By.CSS_SELECTOR, value='#shozaichiBlock > div > table')
        serviceName = tableTexts.find_element(by=By.ID, value='serviceName').text
        
        # 郵便番号と住所
        postalcode_ = tableTexts.find_elements(by=By.TAG_NAME, value='div')[1].text
        postalcode = re.search('[0-9]{3}-[0-9]{4}', postalcode_).group()
        address = re.sub('[0-9]{3}-[0-9]{4}', '', postalcode_).replace('〒\u3000','')

        # 電話番号とFAX
        target = '／'
        nums = tableTexts.find_elements(by=By.TAG_NAME, value='div')[3].text
        idx = nums.find(target)
        tel = nums[:idx].replace('Tel：','')
        fax = nums[idx+1:].replace('Fax：','')
        
        datum = {
                '事業所名':name,
                'サービス種別':serviceName,
                '郵便番号':postalcode,
                '住所':address,
                'TEL':tel,
                'FAX':fax
            }
        data.append(datum)

        # 従業員情報     
        tableGroup4 = driver.find_element(by=By.CSS_SELECTOR, value='#tableGroup-4')
        staffInfos4 = tableGroup4.find_elements(by=By.TAG_NAME, value='td')
        tableGroup5 = driver.find_element(by=By.CSS_SELECTOR, value='#tableGroup-5')
        staffInfos5 = tableGroup5.find_elements(by=By.TAG_NAME, value='td')

        # 従業者数テーブルはサービス種別によってセレクターがばらばらなので条件分岐する
        wait
        if serviceName == '居宅介護支援':
            for staffInfo in staffInfos4:
                table.append(staffInfo.text.replace('人',''))

            allOfStaffs =  table[0]
            careMgr_jokin = table[1]
            careMgr_hijokin = table[2]
            shuninCareMgr_jokin = table[3]
            shuninCareMgr_hijokin = table[4]
            careMgrQuit_jokin = table[5]
            careMgrQuit_hijokin = table[6]
            careMgrNurse_jokin = table[7]
            careMgrNurse_hijokin = table[8]
            careMgrkaifuku_jokin = table[9]
            careMgrkaifuku_hijokin = table[10]
            careMgrRate_10yearsExp = table[11]

        elif serviceName == '看護小規模多機能型居宅介護（複合型サービス）':
            for staffInfo in staffInfos5:
                table.append(staffInfo.text.replace('人',''))

            allOfStaffs =  table[0]
            careStaff_jokin = table[1]
            careStaff_hijokin = table[2]
            Nurse_jokin = table[3]
            Nurse_hijokin = table[4]
            careStaffNurseQuit_jokin = table[5]
            careStaffNurseQuit_hijokin = table[6]
            careStaffRate_10yearsExp = table[7]
            nightShift = table[8]

        elif serviceName == '小規模多機能型居宅介護':
            for staffInfo in staffInfos5:
                table.append(staffInfo.text.replace('人',''))

            allOfStaffs =  table[0]
            careStaff_jokin = table[1]
            careStaff_hijokin = table[2]
            Nurse_jokin = table[3]
            Nurse_hijokin = table[4]
            staffQuit_jokin = table[5]
            staffQuit_hijokin = table[6]
            careMgrRate_10yearsExp = table[7]
            nightShift = table[8]

        elif serviceName == '訪問入浴介護':
            for staffInfo in staffInfos4:
                table.append(staffInfo.text.replace('人',''))

            allOfStaffs =  table[0]
            nursePracticalNurse_jokin = table[1]
            nursePracticalNurse_hijokin = table[2]
            nursePracticalNurseQiut_jokin = table[3]
            nursePracticalNurseQiut_hijokin = table[4]
            careStaff_jokin = table[5]
            careStaff_hijokin = table[6]
            careStaffQuit_jokin = table[7]
            careStaffQuit_hijokin = table[8]
            careStaffKaifuku_jokin = table[9]
            careStaffKaifuku_hijokin = table[10]
            careStaffRate_10yearsExp = table[11]

        elif serviceName == '通所介護' or serviceName == '認知症対応型通所介護':
            for staffInfo in staffInfos5:
                table.append(staffInfo.text.replace('人',''))

            allOfStaffs =  table[0]
            Nurse_jokin = table[1]
            Nurse_hijokin = table[2]
            NurseQuit_jokin = table[3]
            NurseQuit_hijokin = table[4]
            careStaff_jokin = table[5]
            careStaff_hijokin = table[6]
            careStaffQuit_jokin = table[7]
            careStaffQuit_hijokin = table[8]
            careStaffRate_10yearsExp = table[9]

        elif serviceName == '地域密着型通所介護':
            for staffInfo in staffInfos5:
                table.append(staffInfo.text.replace('人',''))

            allOfStaffs =  table[0]
            Nurse_jokin = table[1]
            Nurse_hijokin = table[2]
            NurseQuit_jokin = table[3]
            NurseQuit_hijokin = table[4]
            careStaff_jokin = table[5]
            careStaff_hijokin = table[6]
            careStaffQuit_jokin = table[7]
            careStaffQuit_hijokin = table[8]
            careStaffRate_5yearsExp = table[9]

        elif (serviceName == '特定施設入居者生活介護　軽費老人ホーム'
          or serviceName == '短期入所療養介護　介護老人保健施設'
          or serviceName == '介護老人保健施設'
          or serviceName == '地域密着型特定施設入居者生活介護 有料老人ホーム'
          or serviceName == '短期入所療養介護　介護療養型医療施設'
          or serviceName == '介護医療院'
          or serviceName == '短期入所療養介護（介護医療院）'
          or serviceName == '地域密着型特定施設入居者生活介護 軽費老人ホーム'
          or serviceName == '地域密着型特定施設入居者生活介護 サービス付き高齢者住宅'
          or serviceName == '介護療養型医療施設'
          or serviceName == '特定施設入居者生活介護 有料老人ホーム：サービス付き高齢者向け住宅'
          or serviceName == '特定施設入居者生活介護 有料老人ホーム'):

            for staffInfo in staffInfos5:
                table.append(staffInfo.text.replace('人',''))

            allOfStaffs =  table[0]
            Nurse_jokin = table[1]
            Nurse_hijokin = table[2]
            NurseQuit_jokin = table[3]
            NurseQuit_hijokin = table[4]
            careStaff_jokin = table[5]
            careStaff_hijokin = table[6]
            careStaffQuit_jokin = table[7]
            careStaffQuit_hijokin = table[8]
            careStaffRate_10yearsExp = table[9]
            nightShift = table[10]

        elif serviceName == '短期入所生活介護' or serviceName == '地域密着型介護老人福祉施設入所者生活介護' or serviceName == '介護老人福祉施設' or serviceName == '療養通所介護':

            for staffInfo in staffInfos5:
                table.append(staffInfo.text.replace('人',''))

            allOfStaffs =  table[0]
            Nurse_jokin = table[1]
            Nurse_hijokin = table[2]
            NurseQuit_jokin = table[3]
            NurseQuit_hijokin = table[4]
            careStaff_jokin = table[5]
            careStaff_hijokin = table[6]
            careStaffQuit_jokin = table[7]
            careStaffQuit_hijokin = table[8]
            careStaffRate_5yearsExp = table[9]

        elif serviceName == '認知症対応型共同生活介護':
            for staffInfo in staffInfos5:
                table.append(staffInfo.text.replace('人',''))

            allOfStaffs =  table[0]
            planner_jokin = table[1]
            planner_hijokin = table[2]
            careStaff_jokin = table[3]
            careStaff_hijokin = table[4]
            careStaffQuit_jokin = table[5]
            careStaffQuit_hijokin = table[6]
            Nurse_jokin = table[7]
            Nurse_hijokin = table[8]
            careStaffRate_5yearsExp = table[9]
            nightShift = table[10]

        elif serviceName == '福祉用具貸与' or serviceName == '特定福祉用具販売':
            for staffInfo in staffInfos4:
                table.append(staffInfo.text.replace('人',''))

            allOfStaffs =  table[0]
            WelEquipSpecialist_jokin = table[1]
            WelEquipSpecialist_hijokin = table[2]
            WelEquipSpecialistQuit_jokin = table[3]
            WelEquipSpecialistQuit_hijokin = table[4]
            WelEquipSpecialistRate_10yearsExp = table[5]

        elif serviceName == '訪問看護':
            for staffInfo in staffInfos4:
                table.append(staffInfo.text.replace('人',''))

            allOfStaffs =  table[0]
            publicHealthNurse_jokin = table[1]
            publicHealthNurse_hijokin = table[2]
            nursePracticalNurse_jokin = table[3]
            nursePracticalNurse_hijokin = table[4]
            publicHealthNurseNursePracticalNurseQiut_jokin = table[5]
            publicHealthNurseNursePracticalNurseQiut_hijokin = table[6]
            publicHealthNurseNursePracticalNurseQiutRate_5yearsExp = table[7]

        elif serviceName == '訪問介護':
            for staffInfo in staffInfos4:
                table.append(staffInfo.text.replace('人',''))

            allOfStaffs =  table[0]
            homeHelper_jokin = table[1]
            homeHelper_hijokin = table[2]
            homeHelperQuit_jokin = table[3]
            homeHelperQuit_hijokin = table[4]
            homeHelperKaifuku_jokin = table[5]
            homeHelperKaifuku_hijokin = table[6]
            homeHelperRate_10yearsExp = table[7]

        elif serviceName == '定期巡回・随時対応型訪問介護看護':
            for staffInfo in staffInfos4:
                table.append(staffInfo.text.replace('人',''))

            allOfStaffs =  table[0]
            homeHelper_jokin = table[1]
            homeHelper_hijokin = table[2]
            homeHelperQuit_jokin = table[3]
            homeHelperQuit_hijokin = table[4]
            homeHelperKaifuku_jokin = table[5]
            homeHelperKaifuku_hijokin = table[6]
            operator_jokin = table[7]
            operator_hijokin = table[8]
            operatorQuit_jokin = table[9]
            operatorQuit_hijokin = table[10]
            Nurse_jokin = table[11]
            Nurse_hijokin = table[12]
            NurseQuit_jokin = table[13]
            NurseQuit_hijokin = table[14]
            homeHelperRate_10yearsExp = table[15]

        elif serviceName == '夜間対応型訪問介護':
            for staffInfo in staffInfos4:
                table.append(staffInfo.text.replace('人',''))

            allOfStaffs =  table[0]
            homeHelper_jokin = table[1]
            homeHelper_hijokin = table[2]
            homeHelperQuit_jokin = table[3]
            homeHelperQuit_hijokin = table[4]
            homeHelperKaifuku_jokin = table[5]
            homeHelperKaifuku_hijokin = table[6]
            operator_jokin = table[7]
            operator_hijokin = table[8]
            operatorQuit_jokin = table[9]
            operatorQuit_hijokin = table[10]
            homeHelperRate_5yearsExp = table[11]

        elif serviceName == '通所リハビリテーション':
            for staffInfo in staffInfos5:
                table.append(staffInfo.text.replace('人',''))

            allOfStaffs =  table[0]
            PTOTST_jokin = table[1]
            PTOTST_hijokin = table[2]
            PTOTSTQuit_jokin = table[3]
            PTOTSTQuit_hijokin = table[4]
            Nurse_jokin = table[5]
            Nurse_hijokin = table[6]
            NurseQuit_jokin = table[7]
            NurseQuit_hijokin = table[8]
            careStaff_jokin = table[9]
            careStaff_hijokin = table[10]
            careStaffQuit_jokin = table[11]
            careStaffQuit_hijokin = table[12]
            PTOTSTRate_10yearsExp = table[13]

        else:
            for staffInfo in staffInfos4:
                table.append(staffInfo.text.replace('人',''))

            allOfStaffs =  table[0]
            PTOTST_jokin = table[1]
            PTOTST_hijokin = table[2]
            PTOTSTQuit_jokin = table[3]
            PTOTSTQuit_hijokin = table[4]
            homeHelperRate_5yearsExp = table[5]

        staffInfos = {
              '総従業者数':allOfStaffs,
              '介護職員数_常勤':careStaff_jokin,
              '介護職員数_非常勤':careStaff_hijokin,
              '夜勤を行う従業者数':nightShift,
              '介護職員のうち介護福祉士の資格を持つ従業者数_常勤':careStaffKaifuku_jokin,
              '介護職員のうち介護福祉士の資格を持つ従業者数_非常勤':careStaffKaifuku_hijokin,
              '経験年数５年以上の介護職員の割合':careStaffRate_5yearsExp,
              '経験年数１０年以上の介護職員の割合':careStaffRate_10yearsExp,
              '介護職員の退職者数_常勤':careStaffQuit_jokin,
              '介護職員の退職者数_常勤_非常勤':careStaffQuit_hijokin,
              '看護職員数_常勤':Nurse_jokin,
              '看護職員数_非常勤':Nurse_hijokin,
              '看護職員の退職者数_常勤':NurseQuit_jokin,
              '看護職員の退職者数_非常勤':NurseQuit_hijokin,
              '介護職員・看護職員の退職者数_常勤':careStaffNurseQuit_jokin,
              '介護職員・看護職員の退職者数_非常勤':careStaffNurseQuit_hijokin,
              '看護師・准看護師数_常勤':nursePracticalNurse_jokin,
              '看護師・准看護師数_非常勤':nursePracticalNurse_hijokin,
              '看護師・准看護師の退職者数_常勤':nursePracticalNurseQiut_jokin,
              '看護師・准看護師の退職者数_非常勤':nursePracticalNurseQiut_hijokin,
              '訪問介護員等数_常勤':homeHelper_jokin,
              '訪問介護員等数_非常勤':homeHelper_hijokin,
              '訪問介護員等の退職者数_常勤':homeHelperQuit_jokin,
              '訪問介護員等の退職者数_非常勤':homeHelperQuit_hijokin,
              '訪問介護員等のうち介護福祉士の資格を持つ従業者数_常勤':homeHelperKaifuku_jokin,
              '訪問介護員等のうち介護福祉士の資格を持つ従業者数_非常勤':homeHelperKaifuku_hijokin,
              '経験年数５年以上の訪問介護員等の割合':homeHelperRate_5yearsExp,
              '経験年数１０年以上の訪問介護員等の割合':homeHelperRate_10yearsExp,
              '従業者の退職者数_常勤':staffQuit_jokin,
              '従業者の退職者数_非常勤':staffQuit_hijokin,
              'ケアマネジャー数_常勤':careMgr_jokin,
              'ケアマネジャー数_非常勤':careMgr_hijokin,
              'うち主任ケアマネジャー数_常勤':shuninCareMgr_jokin,
              'うち主任ケアマネジャー数_非常勤':shuninCareMgr_hijokin,
              'ケアマネジャーのうち看護師の資格を持つ従業者数_常勤':careMgrNurse_jokin,
              'ケアマネジャーのうち看護師の資格を持つ従業者数_非常勤':careMgrNurse_hijokin,
              'ケアマネジャーのうち介護福祉士の資格を持つ従業者数_常勤':careMgrkaifuku_jokin,
              'ケアマネジャーのうち介護福祉士の資格を持つ従業者数_常勤_非常勤':careMgrkaifuku_hijokin,
              'ケアマネジャーの退職者数_常勤':careMgrQuit_jokin,
              'ケアマネジャーの退職者数_非常勤':careMgrQuit_hijokin,
              '経験年数１０年以上の介護支援専門員の割合':careMgrRate_10yearsExp,
              '計画作成担当者数_常勤':planner_jokin,
              '計画作成担当者数_非常勤':planner_hijokin,
              '福祉用具専門相談員数_常勤':WelEquipSpecialist_jokin,
              '福祉用具専門相談員数_非常勤':WelEquipSpecialist_hijokin,
              '福祉用具専門相談員の退職者数_常勤':WelEquipSpecialistQuit_jokin,
              '福祉用具専門相談員の退職者数_非常勤':WelEquipSpecialistQuit_hijokin,
              '経験年数１０年以上の福祉用具専門相談員の割合':WelEquipSpecialistRate_10yearsExp,
              '保健師数_常勤':publicHealthNurse_jokin,
              '保健師数_非常勤':publicHealthNurse_hijokin,
              '保健師・看護師・准看護師の退職者数_常勤':publicHealthNurseNursePracticalNurseQiut_jokin,
              '保健師・看護師・准看護師の退職者数_非常勤':publicHealthNurseNursePracticalNurseQiut_hijokin,
              '経験年数５年以上の保健師・看護師・准看護師の割合':publicHealthNurseNursePracticalNurseQiutRate_5yearsExp,
              'オペレーター数_常勤':operator_jokin,
              'オペレーター数_非常勤':operator_hijokin,
              'オペレーターの退職者数_常勤':operatorQuit_jokin,
              'オペレーターの退職者数_非常勤':operatorQuit_hijokin,
              '理学療法士・作業療法士・言語聴覚士の数_常勤':PTOTST_jokin,
              '理学療法士・作業療法士・言語聴覚士の数_非常勤':PTOTST_hijokin,
              '理学療法士・作業療法士・言語聴覚士の退職者数_常勤':PTOTSTQuit_jokin,
              '理学療法士・作業療法士・言語聴覚士の退職者数_非常勤':PTOTSTQuit_hijokin,
              '経験年数１０年以上の理学療法士・作業療法士・言語聴覚士の割合':PTOTSTRate_10yearsExp 
              }
        
        staffInfoData.append(staffInfos)
        print(f'{name} 追加')
        
        # ブラウザバック
        driver.back()
    
    wait
    sleep(5)
    driver.find_element(by=By.CSS_SELECTOR, value='#sel_ResultListForm > div.tab-content > nav:nth-child(1) > ul.pagination.justify-content-end.mt-1 > li:nth-child(12) > a').click()
    print(f'{allOfItems}ページ中{i+1}ページ目終了')

In [816]:
driver.quit()

In [ ]:
data_summary = pd.DataFrame(data)
staff_summary = pd.DataFrame(staffInfoData)
resarch = pd.DataFrame(pd.concat([data_summary, staff_summary], axis=1))

In [ ]:
resarch.to_csv('info_2.csv', encoding='utf_8_sig')